### Wednesday, December 13, 2023

Let's play with the newest model, shall we ... this is a brand new model, just out this morning!!

There was some wonkiness with this notebook with moving either the model or the tensors to device which I corrected. This now all runs on the GPU. By the end, there is 15216 MiB on the VRAM.

Nice! This all runs!

[mistralai/Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2)

### Friday, December 1, 2023

docker container start hfpt_Oct28

https://colab.research.google.com/drive/1F2PeWl5FOHv4sjd7XTEu40JjqbFhC3LB?usp=sharing

In [1]:
import time
from datetime import date

startTime = time.time()
todaysDate = date.today()

# Run Date: Wednesday, December 13, 2023
# Run Time: 00:03:18

In [2]:
!ls /home/rob/Data2/huggingface/transformers

models--mistralai--Mistral-7B-Instruct-v0.1  tmpc719_es2  tmpz6ieywz4
models--model_name			     tmpcjh0h7gn  tmpzafytbf_
tmp05dbfcvi				     tmpczxu4e1z  version.txt
tmp0m77pc0x				     tmpfcrmwgx2


In [3]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-v0.1


In [4]:
#  !pip -q install git+https://github.com/huggingface/transformers # need to install from github
# !pip install -q datasets loralib sentencepiece
# !pip -q install bitsandbytes accelerate xformers einops
# !pip -q install langchain

In [5]:
!nvidia-smi

Wed Dec 13 19:55:59 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        On  | 00000000:01:00.0  On |                  N/A |
|  0%   57C    P0              N/A /  70W |    501MiB /  2048MiB |     32%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Mistral 7B Instruct



In [6]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [7]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
# Load the model if it's not in this container
# mistralai/Mistral-7B-Instruct-v0.1
# docker cp /home/rob/Data3/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.1 c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 15GB to c9b676310ea0://home/rob/Data2/huggingface/transformers


In [9]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [10]:
# We don't need to run this
# torch.set_default_device('cuda')

In [11]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.1'
# model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype="auto")

In [13]:

# This does not work ... ALL VRAM is sucked up almost immediately ...
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", )

# This works! 5478 MiB of VRAM ... but then fails when we try to move the model to the device a few cells below ...
# So to use this, you need to comment out any code that attempts to move the model to the gpu .. 
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, load_in_4bit=True )

# This attempts to load the model to CPU RAM
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# This slowly sucks up CPU RAM, taking up about 16gb (4.9gb to 20.9gb), then drops it into the GPU VRAM taking up 14966 MiB
# 2m 0.2s
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)

# This immediately sucks up all the VRAM!
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# This fails on the dtype parameter
# model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.bfloat16)

# Download and load time for ...
# mistralai/Mistral-7B-Instruct-v0.2
# CPU times: user 57.2 s, sys: 1min 16s, total: 2min 13s
# Wall time: 3h 31min 10s

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Now that the model has been downloaded, let's back it up.

In [14]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.1  /home/rob/Data3/huggingface/transformers
# Successfully copied 14.5GB to /home/rob/Data3/huggingface/transformers

# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.2  /home/rob/Data3/huggingface/transformers
# Successfully copied 14.5GB to /home/rob/Data3/huggingface/transformers

In [15]:
text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

# encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

# device = 'cuda'
# model_inputs = encodeds.to(device)
# model.to(device)

model_inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False).to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s>  [INST] Do you have mayonnaise recipes? [/INST]Of course! I'd be happy to share a classic mayonnaise recipe with you:

Ingredients:
- 1 1/2 cups of unsalted butter, at room temperature
- 2 egg yolks
- 2 tsp of Dijon mustard
- 1/4 cup of white wine vinegar
- Salt and pepper to taste

Instructions:

1. Whisk the butter in a large mixing bowl until it's light and fluffy.
2. Crack open the egg yolks and whisk them in a separate bowl.
3. Slowly pour the butter mixture into the egg yolks, whisking constantly to prevent curdling.
4. Once the mixture is well combined, whisk in the Dijon mustard, white wine vinegar, and a pinch of salt and pepper.
5. Continue whisking the mixture for another 5-7 minutes until it's thick and creamy.
6. Taste and adjust seasoning as desired, then transfer t

### Prompt Format
```
<s>[INST] <user_prompt> [/INST]

<assistant_response> </s>

[INST] <user_prompt>[/INST]
```

In [16]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [17]:
def generate(input_text, system_prompt="",max_length=512):
    prompt = f"""<s>[INST]{input_text}[/INST]"""
    # inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)
    # model_inputs = encodeds.to(device)
    # model.to(device)

    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False).to(device)

    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    wrapped_text = wrap_text(text)
    print(wrapped_text)

## CodeGen

In [18]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """
   for i in range(2, n+1):
       if is_prime(i):
           print(i)

def is_prime(n):
   """
   Check if n is prime
   """
   if n <= 1:
       return False
   for i in range(2, int(n**0.5)+1):
       if n % i == 0:
           return False
   return True

print_prime(100)
```

This code defines two functions: `print_prime` and `is_prime`. The `print_prime` function
takes an integer `n` as input and prints all primes between 1 and `n`. The `is_prime`
function takes an integer `n` as input and returns `True` if `n` is prime, and `False`
otherwise.

The `print_prime` function uses a loop to iterate over all integers between 2 and `n`, and
checks if each integer is prime using the `is_prime` function. If an integer is prime, it
is printed to the console.

The `is_prime` function uses a loop to iterate over all integers between 2 and the square
root of `n`, and checks if `n` is divisible by any of these intege

In [19]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """
   if n <= 1:
       return False
   for i in range(2, int(n**0.5)+1):
       if n % i == 0:
           return False
   return True

def prime_numbers(n):
   """
   return a list of prime numbers less than or equal to n
   """
   prime_list = []
   for i in range(2, n+1):
       if detect_prime(i):
           prime_list.append(i)
   return prime_list

# test
print(prime_numbers(100))</s>


## Instruction Answering

In [20]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a lighthouse.

Mathematics is like a lighthouse in many ways. Just as a lighthouse guides ships safely to
harbor by emitting a steady beam of light, mathematics provides a clear and precise set of
rules and principles that guide us through the complexities of the world.

The lighthouse is a towering structure that stands tall and strong, just as mathematics is
a solid foundation that supports our understanding of the world. Both are essential for
navigating through the unknown waters of life.

The lighthouse emits a


In [21]:
generate('Write a detailed analogy between mathematics and a music.',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a music.

Mathematics and music are two of the most fundamental and universal human disciplines.
Both involve patterns, structures, and the manipulation of abstract concepts. In many
ways, they are closely related, and understanding this relationship can help us to better
understand both subjects.

One way to think about the relationship between mathematics and music is to compare them
to two different languages. Just as English and Spanish are two different languages that
share many similarities, mathematics and music are two different disciplines that share
many similarities. Both involve the use of symbols and notation to represent abstract
concepts, and both involve the manipulation of these concepts to create new ideas and
insights.

For example, in mathematics, we use symbols like "x" and "y" to represent variables, and
we use notation like "=" and "+" to represent operations. Similarly, in music, we use
symbols like "C" and "D" to

In [22]:
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the difference between a Llama, Vicuna and an Alpaca?

I'm looking for a new pet and I'm interested in these three animals. I'm not sure what the
differences are between them.

Comment: I'm not sure if this is the right place to ask this question, but I'm not sure if
you're asking about the differences between the animals or the differences between the
breeds.

Comment: I'm asking about the differences between the animals.

Comment: I'm not sure if you're asking about the differences between the animals or the
differences between the breeds.

Comment: I'm asking about the differences between the animals.

Comment: I'm not sure if you're asking about the differences between the animals or the
differences between the breeds.

Comment: I'm asking about the differences between the animals.

Comment: I'm not sure if you're asking about the differences between the animals or the
differences between the breeds.

Comment: I'm asking about the differences between the animals.

Comment: 

In [23]:
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a short email to Sam Altman giving reasons to open source GPT-4.

Subject: Request to Open Source GPT-4

Dear Sam,

I hope this email finds you well. I am writing to request that OpenAI open sources GPT-4.
As a researcher and developer in the field of artificial intelligence, I believe that open
sourcing GPT-4 would have a significant impact on the field and benefit the community as a
whole.

Here are a few reasons why I believe GPT-4 should be open sourced:

1. Advancements in AI research: Open sourcing GPT-4 would allow researchers and developers
to build upon and improve the model, leading to faster advancements in the field.
2. Accessibility: Open sourcing GPT-4 would make it more accessible to researchers and
developers who may not have the resources to develop their own models from scratch.
3. Transparency: Open sourcing GPT-4 would increase transparency in the development and
deployment of AI models, which is important for building trust in the technology.

I understand th

In [24]:
generate('What is the capital of England?',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the capital of England?

London is the capital of England.</s>


In [25]:
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before
answering.

Geoffrey Hinton is a living person who is a computer scientist and cognitive psychologist.
George Washington was a historical figure who lived from 1732 to 1799 and was the first
President of the United States.

The two are separated by a significant time and space gap, and there is no evidence to
suggest that they ever met or communicated with each other. Therefore, it is not possible
for Geoffrey Hinton to have a conversation with George Washington.</s>


In [26]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a story about a Koala playing pool and beating all the camelids.

Once upon a time, in a land far, far away, there was a little Koala named Kiki. Kiki lived
in a tree with her family, and all day long she would play and have fun with her friends.
But one day, Kiki's friends challenged her to a game of pool. They said that they were the
best pool players in the land, and that they could beat any Koala who dared to try.

Kiki was excited to show her friends that she was just as good as they were, so she agreed
to play. But when she got to the pool table, she realized that she had never played pool
before. She had never even seen a pool table before!

But Kiki was not one to give up easily. She asked her friends to teach her how to play,
and they showed her the basics. Kiki practiced for hours, trying to perfect her shots and
learn the rules of the game.

Finally, the day of the big pool tournament arrived. Kiki's friends were all there, ready
to show her who was boss. But when it w

## Chat

In [27]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: Well, have you tried breaking your study sessions into smaller chunks? It's easier
to concentrate for shorter periods of time. Also, make sure you're taking breaks and doing
something enjoyable in between.

 Alice: That's a good idea, I'll give it a try.

 Bob: And don't forget to prioritize your study material. Focus on the most important
topics first and work your way down.

 Alice: Thanks for the advice, I'll


In [28]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: Well, have you tried breaking your study sessions into smaller chunks? It's easier
to concentrate for short periods of time.

 Alice: That's a good idea, I'll give it a try.

 Bob: Also, make sure you're taking breaks in between study sessions. It's important to
rest your mind.

 Alice: I've been doing that, but I still feel like I'm not retaining the information.

 Bob: Have


## GSM8K

In [29]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer the following question by reasoning step by step. The cafeteria had 23 apples. If
they used 20 for lunch, and bought 6 more, how many apple do they have?
Answer: They have 29 apples.

Explanation:

1. The cafeteria started with 23 apples.
2. They used 20 of those apples for lunch, leaving them with 23 - 20 = 3 apples.
3. Then, they bought 6 more apples, adding them to the remaining 3 apples.
4. So, the total number of apples they have now is 3 (remaining from lunch) + 6 (newly
bought) = 29 apples.</s>


In [30]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting.
How much did she earn?
A: $6</s>


In [31]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


A deep-sea monster rises from the waters once every hundred years to feast on a ship and
sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been
built larger over time, so each new ship has twice as many people as the last ship. How
many people were on the ship the monster ate in the first hundred years?
A: 14</s>


In [32]:
endTime = time.time()

elapsedTime = time.strftime("%H:%M:%S", time.gmtime(endTime - startTime))

print(todaysDate.strftime('# Run Date: %A, %B %d, %Y'))
print(f"# Run Time: {elapsedTime}")

# Run Date: Wednesday, December 13, 2023
# Run Time: 00:03:18
